#**Sales Department Efficiency Analysis**

1. Evaluate the effectiveness of individual deal owners and advertising campaigns in terms of the number of processed deals, conversion rate, and total sales amount.

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
pd.set_option('display.max_columns', None) #display all columns (so that they are not hidden with «…»)

In [ ]:
deals = pd.read_pickle('deals_df.pkl')

In [ ]:
deals.head()

,Id,Deal Owner Name,Closing Date,Quality,Stage,Lost Reason,Campaign,SLA,Content,Term,Source,Payment Type,Product,Education Type,Created Time,Course duration,Months of study,Initial Amount Paid,Offer Total Amount,Contact Name,City,Level of Deutsch,_open_deal,Payment Category,Offer Category,_SLA_hours
0,5805028000056864695,Ben Hall,NaT,Unknown,New Lead,Unknown,03.07.23women,0 days 00:00:00,v16,women,Facebook Ads,Unknown,Unknown,Unknown,2024-06-21 15:30:00,0,0,NaN,NaN,5805028000056849495,Unknown,Unknown,True,Unknown,Unknown,0.000000
1,5805028000056859489,Ulysses Adams,NaT,Unknown,New Lead,Unknown,Unknown,0 days 00:00:00,Unknown,Unknown,Organic,Unknown,Web Developer,Morning,2024-06-21 15:23:00,6,0,0.0,2000.0,5805028000056834471,Unknown,Unknown,True,No Payment,Regular Offer,0.000000
2,5805028000056832357,Ulysses Adams,2024-06-21,D - Non Target,Lost,Non target,engwien_AT,0 days 00:26:43,b1-at,21_06_2024,Telegram posts,Unknown,Unknown,Unknown,2024-06-21 14:45:00,0,0,NaN,NaN,5805028000056854421,Unknown,Unknown,False,Unknown,Unknown,0.445278
3,5805028000056824246,Eva Kent,2024-06-21,E - Non Qualified,Lost,Invalid number,04.07.23recentlymoved_DE,0 days 01:00:03.999999999,bloggersvideo14com,recentlymoved,Facebook Ads,Unknown,Unknown,Unknown,2024-06-21 13:32:00,0,0,NaN,NaN,5805028000056889351,Unknown,Unknown,False,Unknown,Unknown,1.001111
4,5805028000056873292,Ben Hall,2024-06-21,D - Non Target,Lost,Non target,discovery_DE,0 days 00:53:12.000000001,website,Unknown,Google Ads,Unknown,Unknown,Unknown,2024-06-21 13:21:00,0,0,NaN,NaN,5805028000056876176,Unknown,Unknown,False,Unknown,Unknown,0.886667


In [ ]:
deals.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21593 entries, 0 to 21592
Data columns (total 26 columns):
 #   Column               Non-Null Count  Dtype          
---  ------               --------------  -----          
 0   Id                   21593 non-null  object         
 1   Deal Owner Name      21593 non-null  category       
 2   Closing Date         14645 non-null  datetime64[ns] 
 3   Quality              21593 non-null  category       
 4   Stage                21593 non-null  category       
 5   Lost Reason          21593 non-null  category       
 6   Campaign             21593 non-null  category       
 7   SLA                  21593 non-null  timedelta64[ns]
 8   Content              21593 non-null  category       
 9   Term                 21593 non-null  category       
 10  Source               21593 non-null  category       
 11  Payment Type         21593 non-null  category       
 12  Product              21593 non-null  category       
 13  Education Type       

In [ ]:
# Just in case: convert Stage to string
deals['Stage'] = deals['Stage'].astype(str)

###**1. Performance of Deal Owners**

**1.1 Aggregation by managers**

In [ ]:
owners = (deals.groupby('Deal Owner Name', observed=False).agg(total_leads=('Id','count'),
               completed_leads=('Stage', lambda s: (s=='Payment Done').sum()),
               failed_leads=('Stage',   lambda s: (s=='Lost').sum()),
               total_sales=('Initial Amount Paid','sum'),
               total_offer_amount=('Offer Total Amount','sum')).reset_index())

owners['conversion_rate'] = np.where(owners['total_leads']>0, owners['completed_leads']/owners['total_leads']*100, np.nan)
owners['avg_ticket'] = np.where(owners['completed_leads']>0, owners['total_sales']/owners['completed_leads'], np.nan)

owners_sorted = owners.sort_values('total_leads', ascending=False)
owners_sorted


,Deal Owner Name,total_leads,completed_leads,failed_leads,total_sales,total_offer_amount,conversion_rate,avg_ticket
5,Charlie Davis,2963,148,2126,445600.0,3822500.0,4.994938,3010.810811
12,Julia Nelson,2241,93,1536,382961.0,3575311.0,4.149933,4117.860215
21,Ulysses Adams,2165,141,1411,541050.0,5117800.0,6.512702,3837.234043
18,Quincy Vincent,1884,65,1379,221601.0,1788900.0,3.450106,3409.246154
17,Paula Underwood,1862,93,1318,326750.0,2842000.0,4.994629,3513.440860
2,Ben Hall,1345,46,887,241700.0,2030800.0,3.420074,5254.347826
15,Nina Scott,1283,46,865,206150.0,1951400.0,3.585347,4481.521739
23,Victor Barnes,1232,44,821,348900.0,2196800.0,3.571429,7929.545455
4,Cara Iverson,1056,27,710,688400.0,912000.0,2.556818,25496.296296
6,Diana Evans,1013,1,884,50450.0,562500.0,0.098717,50450.000000


**Revenue leaders:**

**Ulysses Adams** — $5.1M total sales.

**Charlie Davis** — $3.8M total sales.

**Julia Nelson** — $3.6M total sales.

**1.2 TOP-N managers: bars + conversion line (with labels on the line)**

In [ ]:

palette = {
    "darkgrey": "#3A3A3A",  # Total Deals
    "yellow": "#FFC300",    # Successful Deals
    "green": "#2CA02C",      # Conversion Rate line
    "line": "#B0B0B0",      # оси/рамки
    "char": "#2E2E2E",      # текст
    "bg": "rgba(0,0,0,0)"   # прозрачный фон
}

TOP_N = 15
top_owners = owners_sorted.head(TOP_N).copy()
top_owners['conv_pct'] = top_owners['conversion_rate'].round(1)

y1_max = float(max(top_owners['total_leads'].max(), top_owners['completed_leads'].max())) * 1.15
y2_max = float(max(5.0, top_owners['conv_pct'].max())) * 1.20

fig = go.Figure()

# 1) Total Deals
fig.add_bar(
    name='Total Deals',
    x=top_owners['Deal Owner Name'],
    y=top_owners['total_leads'],
    marker=dict(color=palette["darkgrey"], line=dict(color=palette["line"], width=1.0)),
    opacity=0.35,
    text=top_owners['total_leads'],
    textposition='outside',
    textfont=dict(size=10, color=palette["char"]),
    cliponaxis=False)

# 2) Successful Deals
fig.add_bar(
    name='Successful Deals',
    x=top_owners['Deal Owner Name'],
    y=top_owners['completed_leads'],
    marker=dict(color=palette["yellow"], line=dict(color=palette["line"], width=1.2)),
    opacity=0.95,
    text=top_owners['completed_leads'],
    textposition='outside',
    textfont=dict(size=10, color=palette["char"]),
    cliponaxis=False)

# 3) Conversion Rate
fig.add_trace(go.Scatter(
    x=top_owners['Deal Owner Name'],
    y=top_owners['conv_pct'],
    mode='lines+markers+text',
    name='Conversion Rate (%)',
    line=dict(color=palette["green"], width=2),
    marker=dict(size=8, line=dict(color=palette["line"], width=1)),
    text=top_owners['conv_pct'].astype(str) + '%',
    textposition='top center',
    textfont=dict(size=11, color=palette["char"]),
    yaxis='y2'))

fig.update_layout(
    title=dict(text='Top Managers: Leads, Successful Deals, Conversion', x=0.0,
               font=dict(size=18, color=palette["char"])),
    barmode='overlay',
    xaxis=dict(
        title='Deal Owner',
        tickangle=45,
        showgrid=False, zeroline=False,
        linecolor=palette["line"], tickcolor=palette["line"]),
    yaxis=dict(
        title='Leads',
        range=[0, y1_max],
        showgrid=False, zeroline=False,
        linecolor=palette["line"], tickcolor=palette["line"]),
    yaxis2=dict(
        title='Conversion Rate (%)',
        overlaying='y',
        side='right',
        range=[0, y2_max],
        showgrid=False, zeroline=False),
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="left", x=0),
    font=dict(family="Arial", color=palette["char"]),
    paper_bgcolor=palette["bg"],
    plot_bgcolor=palette["bg"],
    margin=dict(l=70, r=70, t=70, b=90))

fig.show()


**1.3 TOP by revenue and average check (quick ranking)**

In [ ]:
top_sales = owners.sort_values('total_sales', ascending=False).head(10)[['Deal Owner Name','total_leads','completed_leads','conversion_rate','total_sales','avg_ticket']].copy()
top_sales['conversion_rate'] = top_sales['conversion_rate'].round(1)
top_sales['avg_ticket'] = top_sales['avg_ticket'].round(0)
top_sales


,Deal Owner Name,total_leads,completed_leads,conversion_rate,total_sales,avg_ticket
4,Cara Iverson,1056,27,2.6,688400.0,25496.0
21,Ulysses Adams,2165,141,6.5,541050.0,3837.0
5,Charlie Davis,2963,148,5.0,445600.0,3011.0
12,Julia Nelson,2241,93,4.1,382961.0,4118.0
23,Victor Barnes,1232,44,3.6,348900.0,7930.0
17,Paula Underwood,1862,93,5.0,326750.0,3513.0
2,Ben Hall,1345,46,3.4,241700.0,5254.0
18,Quincy Vincent,1884,65,3.5,221601.0,3409.0
15,Nina Scott,1283,46,3.6,206150.0,4482.0
16,Oliver Taylor,163,50,30.7,152650.0,3053.0


In [ ]:
vol_thr = 100  # порог по объёму для честного сравнения конверсии

top_owners_by_leads = owners_sorted.head(3)[['Deal Owner Name','total_leads','completed_leads','conversion_rate','total_sales','avg_ticket']].copy()

top_owners_by_conv = (owners[owners['total_leads']>=vol_thr]
                      .sort_values('conversion_rate', ascending=False).head(3)[['Deal Owner Name','total_leads','completed_leads','conversion_rate','total_sales','avg_ticket']].copy())

for df in (top_owners_by_leads, top_owners_by_conv):
    df['conversion_rate'] = df['conversion_rate'].round(1)
    df['avg_ticket'] = df['avg_ticket'].round(0)

print("=== Managers — Top by Leads ===\n", top_owners_by_leads.to_string(index=False))
print("\n=== Managers — Top by Conversion (volume ≥ 100) ===\n", top_owners_by_conv.to_string(index=False))


=== Managers — Top by Leads ===
 Deal Owner Name  total_leads  completed_leads  conversion_rate  total_sales  avg_ticket
  Charlie Davis         2963              148              5.0     445600.0      3011.0
   Julia Nelson         2241               93              4.1     382961.0      4118.0
  Ulysses Adams         2165              141              6.5     541050.0      3837.0

=== Managers — Top by Conversion (volume ≥ 100) ===
 Deal Owner Name  total_leads  completed_leads  conversion_rate  total_sales  avg_ticket
  Oliver Taylor          163               50             30.7     152650.0      3053.0
   Kevin Parker          574               40              7.0      72850.0      1821.0
  Ulysses Adams         2165              141              6.5     541050.0      3837.0


**Leads volume:**

- Charlie Davis (2,963 leads) and Julia Nelson (2,241 leads) лидеры по количеству обработанных сделок.

- Однако их conversion rate сравнительно низкий (4–5%), что снижает общую эффективность.

**High conversion:**

- Oliver Taylor показывает лучший результат по конверсии (30.7%) при сравнительно небольшом объёме (163 leads).

- Kevin Parker (6.9%) и Ulysses Adams (6.5%) также выше среднего уровня (~4–5%).

**Weak performance:**

- Несколько сотрудников имеюточень низкую конверсию 1%.

###**2) Campaign effectiveness (by sales department)**

**2.1 Aggregation by campaigns**

In [ ]:
# Optionally exclude 'Unknown' campaigns to focus on manageable ones
d_camp = deals[deals['Campaign'].astype(str)!='Unknown'].copy()

camp = (d_camp.groupby('Campaign', observed=False).agg(total_leads=('Id','count'),
             completed_leads=('Stage', lambda s: (s=='Payment Done').sum()),
             failed_leads=('Stage',   lambda s: (s=='Lost').sum()),
             total_sales=('Initial Amount Paid','sum'),
             total_offer_amount=('Offer Total Amount','sum')).reset_index())

camp['conversion_rate'] = np.where(camp['total_leads']>0,
                                   camp['completed_leads']/camp['total_leads']*100, np.nan)

camp_sorted = camp.sort_values('total_leads', ascending=False)
camp_sorted.head(10)


,Campaign,total_leads,completed_leads,failed_leads,total_sales,total_offer_amount,conversion_rate
119,performancemax_digitalmarkt_ru_DE,2653,112,2335,422900.0,3333600.0,4.221636
153,youtube_shorts_DE,1635,53,1322,253300.0,2018100.0,3.241590
18,12.07.2023wide_DE,1575,48,1314,277000.0,2064900.0,3.047619
2,02.07.23wide_DE,975,52,781,195950.0,1661100.0,5.333333
5,04.07.23recentlymoved_DE,750,31,597,143100.0,1135600.0,4.133333
4,03.07.23women,612,31,474,120550.0,1173698.0,5.065359
60,Dis_DE,581,30,506,117950.0,1228800.0,5.163511
8,07.07.23LAL_DE,542,28,377,126800.0,1056500.0,5.166052
19,12.09.23interests_Uxui_DE,531,27,421,140450.0,1221300.0,5.084746
40,24.09.23retargeting_DE,479,17,373,116650.0,980400.0,3.549061


In [ ]:
top_camp_by_leads = camp_sorted.head(3)[['Campaign','total_leads','completed_leads','conversion_rate','total_sales']].copy()
top_camp_by_conv = (camp[camp['total_leads']>=vol_thr]
                    .sort_values('conversion_rate', ascending=False).head(3)[['Campaign','total_leads','completed_leads','conversion_rate','total_sales']].copy())

for df in (top_camp_by_leads, top_camp_by_conv):
    df['conversion_rate'] = df['conversion_rate'].round(1)

print("\n=== Campaigns — Top by Leads ===\n", top_camp_by_leads.to_string(index=False))
print("\n=== Campaigns — Top by Conversion (volume ≥ 100) ===\n", top_camp_by_conv.to_string(index=False))



=== Campaigns — Top by Leads ===
                          Campaign  total_leads  completed_leads  conversion_rate  total_sales
performancemax_digitalmarkt_ru_DE         2653              112              4.2     422900.0
                youtube_shorts_DE         1635               53              3.2     253300.0
                12.07.2023wide_DE         1575               48              3.0     277000.0

=== Campaigns — Top by Conversion (volume ≥ 100) ===
            Campaign  total_leads  completed_leads  conversion_rate  total_sales
brand_search_eng_DE          168               15              8.9     114050.0
    02.07.23wide_DE          975               52              5.3     195950.0
     07.07.23LAL_DE          542               28              5.2     126800.0


**By leads volume:**

- performancemax_digitalmarkt_ru_DE (2,653 leads) – главный генератор лидов, но низкая конверсия (4.2%).

- youtube_shorts_DE (1,635 leads) и wide_DE (1,575 leads) — также лидеры по количеству, но конверсия лишь 3–3.2%.

**By conversion rate (≥100 leads):**

- brand_search_eng_DE (8.9%), wide_DE (02.07.23) (5.3%), LAL_DE (5.2%).

- Эти кампании привлекают меньше лидов, но качество выше.

**Recommendations**

увеличить бюджеты для brand_search_eng_DE, LAL_DE.

пересмотреть креативы и аудиторию для performancemax и youtube_shorts_DE.

снять нагрузку с Davis/Nelson и передать часть лидов более эффективным сотрудникам.

провести обучение для менеджеров с низкой конверсией.

использовать успешные кейсы для увеличения среднего чека в других командах.